In [11]:
import numpy as np
from scipy.stats import entropy
import pandas as pd

In [4]:
!samtools view data/CLIP-35L33G.bam | head

SRR458758.23028115	0	chr1	3056473	0	20M	*	0	0	GAATGGAAGTTCAAGGATCT	HHGHHGE@GGHHED?GEGDG	MD:Z:20	NH:i:40	HI:i:1	NM:i:0	SM:i:0	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:-
SRR458758.23879202	0	chr1	3056473	0	20M	*	0	0	GAATGGAAGTTCAAGGATCT	GBGGEEAAF=CEEDEGBGG>	MD:Z:20	NH:i:40	HI:i:1	NM:i:0	SM:i:0	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:-
SRR458758.23893532	0	chr1	3056473	0	20M	*	0	0	GAATGGAAGTTCAAGGATCT	G@?GGE=EFBDDGGGGD@GG	MD:Z:20	NH:i:40	HI:i:1	NM:i:0	SM:i:0	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:-
SRR458758.26501547	0	chr1	3056473	0	20M	*	0	0	GAATGGAAGTTCAAGGATCT	IIIHIIIHIGHIIIHEHIHH	MD:Z:20	NH:i:40	HI:i:1	NM:i:0	SM:i:0	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:-
SRR458758.685997	16	chr1	3059006	0	22M	*	0	0	TTCATTTACAGAATGGAATACT	EG7:776DGGGBD<GGEBGEG@	MD:Z:22	NH:i:30	HI:i:1	NM:i:0	SM:i:0	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:+
SRR458758.12045118	16	chr1	3059006	0	22M	*	0	0	TTCATTTACAGAATGGAATACT	IFGIDDIHIHEIIIIIIIIHBI	MD:Z:22	NH:i:30	HI:i:1	NM:i:0	SM:i:0	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:+
SRR458758.15313921	16	chr1	3059006	0	22M	*	0	0	TTCATTTAC

In [5]:
# Position별로 mapping된 base 확인을 위해 mpileup
# base의 비율만 보면 되니까 reference는 따로 필요없음

!samtools mpileup data/CLIP-35L33G.bam > data/CLIP-35L33G.pileup

[mpileup] 1 samples in 1 input files


In [6]:
!head data/CLIP-35L33G.pileup

chr1	3056473	N	4	^!G^!G^!G^!G	HGGI
chr1	3056474	N	4	AAAA	HB@I
chr1	3056475	N	4	AAAA	GG?I
chr1	3056476	N	4	TTTT	HGGH
chr1	3056477	N	4	GGGG	HEGI
chr1	3056478	N	4	GGGG	GEEI
chr1	3056479	N	4	AAAA	EA=I
chr1	3056480	N	4	AAAA	@AEH
chr1	3056481	N	4	GGGG	GFFI
chr1	3056482	N	4	TTTT	G=BG


In [8]:
# Read count가 50보다 큰 position만 filtering

!awk '$4 > 50 { print $0; }' data/CLIP-35L33G.pileup > data/CLIP-35L33G_filtered.pileup

In [10]:
# pileup file의 크기가 너무 커서, filtering 이전의 file은 삭제

!rm -rf data/CLIP-35L33G.pileup

In [9]:
!head data/CLIP-35L33G_filtered.pileup

chr1	3222722	N	124	CCCAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A	HCGGHEIHHHIBDDIIIGIG>GIHGDHHDDIAEIEEBIGIBGG8GIH@HEIHDIHDIIGHFFGFIIHI@HIGHIIIIIIIIGGBIBHHIHDGIIHBIF=GDDHIHGIHDHIHGBGBFGGHHGGF
chr1	3222723	N	146	GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G	E>HIHFIIHHIGDFIII<IHBIIHGDHEB?I@HIEIBIII;>G;GII@@HIHGIIEIIGIH0DIIIHIDHIBGIIIIIIIIIF?IBHHIHDGIIHGIBGEHDEI1HGHHEHFIGB@GH>GHDE>IGHIBHIEGGHGBGHGIGIDII
chr1	3222724	N	155	AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A^!A

In [14]:
# pileup file에서 사용할 column만 dataframe으로 불러옴

df = pd.read_csv("data/CLIP-35L33G_filtered.pileup", sep="\t", usecols=[0, 1, 3, 4], names=["chrom", "pos", "count", "basereads"])
df

,chrom,pos,count,basereads
0,chr1,3222722,124,CCCAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A...
1,chr1,3222723,146,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...
2,chr1,3222724,155,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
3,chr1,3222725,157,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...
4,chr1,3222726,156,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...
...,...,...,...,...
24696039,MU069435.1,1564,305,>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...
24696040,MU069435.1,1565,297,>>>>AAAAAAAAAAAA$AA$AA$A$AA$AA$A$AAAAAAAAAAAAA...
24696041,MU069435.1,1566,289,>>>>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...
24696042,MU069435.1,1567,291,>>>>T$T$T$T$T$T$T$T$T$T$T$T$T$T$T$TTTTTTTTTT$T...


In [26]:
# Read count가 50 넘는 position만 잘 filtering되었는지 확인

(df["count"] > 50).all()

True

In [27]:
# match & substitution만 남김

import re
toremove = re.compile('[<>$*#^]')
df["matches"] = df["basereads"].apply(lambda x: toremove.sub('', x))
df

,chrom,pos,count,basereads,matches
0,chr1,3222722,124,CCCAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A...,CCCAAAAAAAAAAAAAAAAAAAAAAAAA!A!A!A!A!A!A!A!A!A...
1,chr1,3222723,146,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...
2,chr1,3222724,155,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
3,chr1,3222725,157,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...
4,chr1,3222726,156,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...
...,...,...,...,...,...
24696039,MU069435.1,1564,305,>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...
24696040,MU069435.1,1565,297,>>>>AAAAAAAAAAAA$AA$AA$A$AA$AA$A$AAAAAAAAAAAAA...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
24696041,MU069435.1,1566,289,>>>>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...
24696042,MU069435.1,1567,291,>>>>T$T$T$T$T$T$T$T$T$T$T$T$T$T$T$TTTTTTTTTT$T...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...


In [42]:
# 모두 대문자로 변경
df["matches"] = df.matches.str.upper()
df

,chrom,pos,count,basereads,matches,A,T,C,G
0,chr1,3222722,124,CCCAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A...,CCCAAAAAAAAAAAAAAAAAAAAAAAAA!A!A!A!A!A!A!A!A!A...,0.975806,0.000000,0.024194,0.000000
1,chr1,3222723,146,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,0.000000,0.000000,0.000000,1.000000
2,chr1,3222724,155,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,1.000000,0.000000,0.000000,0.000000
3,chr1,3222725,157,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,0.031847,0.000000,0.000000,0.968153
4,chr1,3222726,156,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,0.352564,0.000000,0.128205,0.519231
...,...,...,...,...,...,...,...,...,...
24696039,MU069435.1,1564,305,>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,0.000000,1.000000,0.000000,0.000000
24696040,MU069435.1,1565,297,>>>>AAAAAAAAAAAA$AA$AA$A$AA$AA$A$AAAAAAAAAAAAA...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,1.000000,0.000000,0.000000,0.000000
24696041,MU069435.1,1566,289,>>>>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,0.000000,0.003509,0.996491,0.000000
24696042,MU069435.1,1567,291,>>>>T$T$T$T$T$T$T$T$T$T$T$T$T$T$T$TTTTTTTTTT$T...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,0.000000,1.000000,0.000000,0.000000


In [43]:
# Count bases per position

for base in ["A", "T", "C", "G"]:
    df[base] = df.matches.str.count(base)

df

,chrom,pos,count,basereads,matches,A,T,C,G
0,chr1,3222722,124,CCCAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A...,CCCAAAAAAAAAAAAAAAAAAAAAAAAA!A!A!A!A!A!A!A!A!A...,121,0,3,0
1,chr1,3222723,146,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,0,0,0,146
2,chr1,3222724,155,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,155,0,0,0
3,chr1,3222725,157,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,5,0,0,152
4,chr1,3222726,156,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,55,0,20,81
...,...,...,...,...,...,...,...,...,...
24696039,MU069435.1,1564,305,>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,0,301,0,0
24696040,MU069435.1,1565,297,>>>>AAAAAAAAAAAA$AA$AA$A$AA$AA$A$AAAAAAAAAAAAA...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,293,0,0,0
24696041,MU069435.1,1566,289,>>>>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,0,1,284,0
24696042,MU069435.1,1567,291,>>>>T$T$T$T$T$T$T$T$T$T$T$T$T$T$T$TTTTTTTTTT$T...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,0,287,0,0


In [45]:
df[["A", "T", "C", "G"]] = df[["A", "T", "C", "G"]].div(df[["A", "T", "C", "G"]].sum(axis=1), axis=0)
df

,chrom,pos,count,basereads,matches,A,T,C,G
0,chr1,3222722,124,CCCAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A...,CCCAAAAAAAAAAAAAAAAAAAAAAAAA!A!A!A!A!A!A!A!A!A...,0.975806,0.000000,0.024194,0.000000
1,chr1,3222723,146,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,0.000000,0.000000,0.000000,1.000000
2,chr1,3222724,155,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,1.000000,0.000000,0.000000,0.000000
3,chr1,3222725,157,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,0.031847,0.000000,0.000000,0.968153
4,chr1,3222726,156,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,0.352564,0.000000,0.128205,0.519231
...,...,...,...,...,...,...,...,...,...
24696039,MU069435.1,1564,305,>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,0.000000,1.000000,0.000000,0.000000
24696040,MU069435.1,1565,297,>>>>AAAAAAAAAAAA$AA$AA$A$AA$AA$A$AAAAAAAAAAAAA...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,1.000000,0.000000,0.000000,0.000000
24696041,MU069435.1,1566,289,>>>>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,0.000000,0.003509,0.996491,0.000000
24696042,MU069435.1,1567,291,>>>>T$T$T$T$T$T$T$T$T$T$T$T$T$T$T$TTTTTTTTTT$T...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,0.000000,1.000000,0.000000,0.000000


In [47]:
# Shannon entropy 계산 

prop = df[["A", "T", "C", "G"]].to_numpy()
df["entropy"] = -np.sum(prop * np.log(prop + 1e-9), axis=1)
df

,chrom,pos,count,basereads,matches,A,T,C,G,entropy
0,chr1,3222722,124,CCCAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A...,CCCAAAAAAAAAAAAAAAAAAAAAAAAA!A!A!A!A!A!A!A!A!A...,0.975806,0.000000,0.024194,0.000000,1.139389e-01
1,chr1,3222723,146,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,0.000000,0.000000,0.000000,1.000000,-1.000000e-09
2,chr1,3222724,155,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,1.000000,0.000000,0.000000,0.000000,-1.000000e-09
3,chr1,3222725,157,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,0.031847,0.000000,0.000000,0.968153,1.411055e-01
4,chr1,3222726,156,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,0.352564,0.000000,0.128205,0.519231,9.712127e-01
...,...,...,...,...,...,...,...,...,...,...
24696039,MU069435.1,1564,305,>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,0.000000,1.000000,0.000000,0.000000,-1.000000e-09
24696040,MU069435.1,1565,297,>>>>AAAAAAAAAAAA$AA$AA$A$AA$AA$A$AAAAAAAAAAAAA...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,1.000000,0.000000,0.000000,0.000000,-1.000000e-09
24696041,MU069435.1,1566,289,>>>>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,0.000000,0.003509,0.996491,0.000000,2.333590e-02
24696042,MU069435.1,1567,291,>>>>T$T$T$T$T$T$T$T$T$T$T$T$T$T$T$TTTTTTTTTT$T...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,0.000000,1.000000,0.000000,0.000000,-1.000000e-09


In [53]:
# Shannon entropy 0.8 이상인 position filtering

pos_df = df[df.entropy >= 0.8]
pos_df

,chrom,pos,count,basereads,matches,A,T,C,G,entropy
4,chr1,3222726,156,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,0.352564,0.000000,0.128205,0.519231,0.971213
46,chr1,3224576,168,CCCG*GGGG*GGGGC*GGGGGGGGGG**GGGGGCTCTGCCCCCCCG...,CCCGGGGGGGGGCGGGGGGGGGGGGGGGCTCTGCCCCCCCGCCCGG...,0.000000,0.075949,0.335443,0.588608,0.874139
166,chr1,4329962,55,<<ccccccccccccc$tttc$c$c$ttttc$ttcggcgggcccccc...,CCCCCCCCCCCCCTTTCCCTTTTCTTCGGCGGGCCCCCCCCCCCTT...,0.000000,0.320755,0.584906,0.094340,0.901133
861,chr1,4841534,272,<<<AGAAGAAAAGGAAGAAAAAAAGAAAAGGATTTTTAAAAAAAAA...,AGAAGAAAAGGAAGAAAAAAAGAAAAGGATTTTTAAAAAAAAAAAA...,0.537879,0.113636,0.113636,0.234848,1.168064
1058,chr1,4846654,691,<<<ccccccccccccccccccccccccttttttccccctccccccc...,CCCCCCCCCCCCCCCCCCCCCCCCTTTTTTCCCCCTCCCCCCCCCC...,0.004392,0.263543,0.629575,0.102489,0.900067
...,...,...,...,...,...,...,...,...,...,...
24695242,chrM,15317,80,GGGGGTCTTTTTTCTTTGGTCCTTTTGTTCGTTTTTTGTGTTCTCT...,GGGGGTCTTTTTTCTTTGGTCCTTTTGTTCGTTTTTTGTGTTCTCT...,0.000000,0.562500,0.212500,0.225000,0.988388
24695339,chrM,15414,90,tt$tttgattttttt$gtt$tgtattttttgattagttt$gat$aa...,TTTTTGATTTTTTTGTTTGTATTTTTTGATTAGTTTGATAAATTTT...,0.388889,0.544444,0.000000,0.066667,0.878844
24695613,MU069435.1,386,217,TT*****GG*G*GGTGTG*G**TG*TG**TTGTTTTG**GGGTCCC...,TTGGGGGTGTGGTGTGTTGTTTTGGGGTCCCGTGTGCGCGGATTGC...,0.032680,0.392157,0.111111,0.464052,1.079309
24695801,MU069435.1,581,2854,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,0.004205,0.128241,0.229502,0.638052,0.910884
